In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("cleaned_output.csv")

# Display the first few rows to understand the structure
print(df.head())

                                            topic_id  \
0  https://guides.loc.gov/chronicling-america-190...   
1  https://guides.loc.gov/chronicling-america-190...   
2  https://guides.loc.gov/chronicling-america-190...   
3  https://guides.loc.gov/chronicling-america-190...   
4  https://guides.loc.gov/chronicling-america-190...   

                                           result_id  rel  \
0  https://www.loc.gov/resource/sn87093407/1904-0...    1   
1  https://www.loc.gov/resource/sn99063957/1904-0...    1   
2  https://www.loc.gov/resource/sn85052116/1904-0...    1   
3  https://www.loc.gov/resource/sn99063957/1904-0...    1   
4  https://www.loc.gov/resource/sn85066387/1904-0...    1   

                                                text  \
0  REPUBLICAN CONVENTION GETS DOWN TO BUSINESS O....   
1  NATIONAL REPUBLICAN CONVENTION Fairbanks for V...   
2  Convention Hall Chicago Juno 23 This the third...   
3  ROOSEVELT AND FAIRBANKS Republican National Co...   
4  ROOSEVELT WIL

In [ ]:
import requests

OLLAMA_API_URL = "http://localhost:11434/api/generate"

def query_llama(text):
    """
    Sends the text to Mistral 7B via Ollama and extracts the predicted date.
    """
    notes = """
                1. The text is from chrnocling america ocr extracted text. So it may contain noise and you are expected to understand with error.
                2. Only provide the date or year or whatever which is only related to the timeperiods.
                3. Consider only the provided text, please don't use your existing knowledge that you got pretrained on when giving the dates.
                4. Do not give any summaries or explantions or any other information.
                5. If the date is not present in the text, just say or write 'No timeline found'.
            """
    prompt = f"What is the exact date of the major event being discussed in the text? Please follow these {notes} and Just give me the date. NOTHING ELSE! The text is - {text}"

    try:
        # Send the request to Ollama
        response = requests.post(
            OLLAMA_API_URL,
            json={
                "model": "llama3",
                "prompt": prompt,
                "stream": False
            }
        )

        # Parse the response
        if response.status_code == 200:
            result = response.json()
            predicted_date = result["response"].strip()  
            return predicted_date
        else:
            print(f"Error: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

In [3]:
# Add a new column for predicted dates
df['predicted_Date'] = None

# Loop through each record and extract the date
for index, row in df.iterrows():
    text = row['text']
    if row['rel'] == 0: pass
    predicted_date = query_llama(text)
    print(f"Predicted date for index {index}: {predicted_date}")
    df.at[index, 'predicted_date'] = predicted_date

# Save the updated DataFrame to a new CSV file
df.to_csv("output_with_predictions.csv", index=False)

print("Processing complete. Updated CSV saved as 'output_with_predictions.csv'.")

Predicted date for index 0: The exact date mentioned in the text is:

* June 1: U.S. Senator Fairbanks is mentioned as being sure to get the nomination for vice president.
* June 11: The mention of a "flurry" in Seattle tideland lands, which is believed to be due to E.H. Harriman's plan to gain terminal grounds in Seattle before starting work on his new line from Portland to Puget Sound.

Note that there are other dates mentioned throughout the text as well, but these are the specific dates provided.
Predicted date for index 1: June 21
Predicted date for index 2: June 23, 1904
Predicted date for index 3: June 20, 1904
Predicted date for index 4: The exact date of the major event being discussed in the text is June 13.
Predicted date for index 5: July 27
Predicted date for index 6: The exact date of the major event being discussed in the text is not specified. However, it appears to be a speech given by Mr. Williams, an unnamed speaker, at a Democratic national convention or similar eve

In [5]:
# Define a function to calculate accuracy
def calculate_accuracy(df):
    """
    Compares the ground truth (`date`) with the `predicted_Date` and calculates accuracy.
    """
    correct_predictions = 0
    total_predictions = len(df)

    for index, row in df.iterrows():
        ground_truth = row['date'].strip()
        predicted_date = row['predicted_Date']

        if predicted_date and ground_truth == predicted_date:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(df)
print(f"Accuracy: {accuracy * 100:.2f}%")

AttributeError: 'float' object has no attribute 'strip'